In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

In [12]:
model=models.vgg19(pretrained=True).features
print(model) #0,5,10,19,28

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [13]:
class vgg(nn.Module):
  def __init__(self):
    super(vgg,self).__init__()
    self.selectedconvlayer=[0,5,10,19,28]
    self.model=models.vgg19(pretrained=True).features[:29]
  
  def forward(self,i):
    features=[]
    for num,layer in enumerate(self.model):
      i=layer(i)
      if num in self.selectedconvlayer:
        features.append(i)
    return features

In [14]:
def img_load(img_name):
  img=Image.open(img_name)
  img=loader(img).unsqueeze(0)
  return img.to(device)

In [15]:
device=torch.device("cuda" if torch.cuda.is_available else "cpu")
img_size=512

In [16]:
loader=transforms.Compose(
    [
     transforms.Resize((img_size,img_size)),
     transforms.ToTensor()
    ]
)

In [17]:
original_img=img_load('Italy.jpg')
style_img=img_load('starrynights.jpg')
model=vgg().to(device).eval()
gen=original_img.clone().requires_grad_(True)

In [18]:
#Hyperparameters
total_steps=8000
learning_rate=0.001
alpha=1
beta=0.01
loss=0

In [19]:
optimizer=optim.Adam([gen],lr=learning_rate)

In [20]:
for step in range(total_steps):
  generated_f=model(gen)
  original_f=model(original_img)
  style_f=model(style_img)
  loss_original=0
  loss_style=0
  for (gen_f,ori_f,st_f) in zip(generated_f,original_f,style_f):
    _,channel,height,width=gen_f.shape
    loss_original+=torch.mean((gen_f-ori_f)**2)
    G=gen_f.view(channel,height*width).mm(gen_f.view(channel,height*width).t())
    A=st_f.view(channel,height*width).mm(st_f.view(channel,height*width).t())
    loss_style+=torch.mean((G-A)**2)
  loss=alpha*loss_original+beta*loss_style
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if(step%200==0):
    print('Total Loss: ',loss)
    save_image(gen,"final.png")


Total Loss:  tensor(2138951.7500, device='cuda:0', grad_fn=<AddBackward0>)
Total Loss:  tensor(111787.3672, device='cuda:0', grad_fn=<AddBackward0>)
Total Loss:  tensor(50003.3438, device='cuda:0', grad_fn=<AddBackward0>)
Total Loss:  tensor(28569.4141, device='cuda:0', grad_fn=<AddBackward0>)
Total Loss:  tensor(20005.2637, device='cuda:0', grad_fn=<AddBackward0>)
Total Loss:  tensor(15312.0986, device='cuda:0', grad_fn=<AddBackward0>)
Total Loss:  tensor(12145.7881, device='cuda:0', grad_fn=<AddBackward0>)
Total Loss:  tensor(9865.4824, device='cuda:0', grad_fn=<AddBackward0>)
Total Loss:  tensor(8185.5215, device='cuda:0', grad_fn=<AddBackward0>)
Total Loss:  tensor(6926.1934, device='cuda:0', grad_fn=<AddBackward0>)
Total Loss:  tensor(5967.8262, device='cuda:0', grad_fn=<AddBackward0>)
Total Loss:  tensor(5227.4331, device='cuda:0', grad_fn=<AddBackward0>)
Total Loss:  tensor(4645.6313, device='cuda:0', grad_fn=<AddBackward0>)
Total Loss:  tensor(4178.2715, device='cuda:0', grad_f